In [2]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

In [3]:
class MnistBags(Dataset):
    def __init__(self, df, mean_bag_length=10, var_bag_length=1, num_bag=1000, seed=7, train=True):
        self.mean_bag_length = mean_bag_length
        self.var_bag_length = var_bag_length
        self.num_bag = num_bag
        self.seed = seed
        self.train = train

        self.r = np.random.RandomState(seed)


        if self.train:
            self.train_bags_list, self.train_labels_list = self._form_bags()
        else:
            self.test_bags_list, self.test_labels_list = self._form_bags()
    def get_grouped_data(self, df):
        
    def _form_bags(self, df):
        bags_list = []
        labels_list = []
        valid_bags_counter = 0
        label_of_last_bag = 0

        for batch_data in loader:
            numbers = batch_data[0]
            labels = batch_data[1]

        
        while valid_bags_counter < self.num_bag:
            bag_length = np.int(self.r.normal(
                self.mean_bag_length, self.var_bag_length, 1))
            if bag_length < 1:
                bag_length = 1
            indices = torch.LongTensor(
                self.r.randint(0, self.ds_count, bag_length))
            labels_in_bag = labels[indices]

            if (self.target_number in labels_in_bag) and (label_of_last_bag == 0):
                labels_in_bag = labels_in_bag >= self.target_number
                labels_list.append(labels_in_bag)
                bags_list.append(numbers[indices])
                label_of_last_bag = 1
                valid_bags_counter += 1
            elif label_of_last_bag == 1:
                index_list = []
                bag_length_counter = 0
                while bag_length_counter < bag_length:
                    index = torch.LongTensor(
                        self.r.randint(0, self.ds_count, 1))
                    label_temp = labels[index]
                    if label_temp.numpy()[0] != self.target_number:
                        index_list.append(index)
                        bag_length_counter += 1

                index_list = np.array(index_list)
                labels_in_bag = labels[index_list]
                labels_in_bag = labels_in_bag >= self.target_number
                labels_list.append(labels_in_bag)
                bags_list.append(numbers[index_list])
                label_of_last_bag = 0
                valid_bags_counter += 1
            else:
                pass

        return bags_list, labels_list

    def __len__(self):
        if self.train:
            return len(self.train_labels_list)
        else:
            return len(self.test_labels_list)

    def __getitem__(self, index):
        if self.train:
            bag = self.train_bags_list[index]
            label = [max(self.train_labels_list[index]),
                     self.train_labels_list[index]]
        else:
            bag = self.train_bags_list[index]
            label = [max(self.train_labels_list[index]),
                     self.train_labels_list[index]]

        return bag, label

In [26]:
df = pd.read_csv('/mnt/d/uczelnia/magister/TCGA_breast_patch/metadata.csv', index_col=0)
df['patient_id'] = df['file_name'].apply(lambda x: x.split('/')[6])

In [27]:
df.set_index(['patient_id', 'file_name'], inplace=True)

In [28]:
df.loc['TCGA-3C-AALI-01A-01-T_R1'].loc['/mnt/d/uczelnia/magister/TCGA_breast_patch/TCGA-3C-AALI-01A-01-T_R1/patch_10_10.png']

label    hr+her2+
Name: /mnt/d/uczelnia/magister/TCGA_breast_patch/TCGA-3C-AALI-01A-01-T_R1/patch_10_10.png, dtype: object

In [33]:
df.index.get_level_values(0).unique()

Index(['TCGA-3C-AALI-01A-01-T_R1', 'TCGA-3C-AALI-01A-01-T_R2',
       'TCGA-5L-AAT0-01A-01-T_R1', 'TCGA-5L-AAT0-01A-01-T_R2',
       'TCGA-A1-A0SE-01A-01-B_R1', 'TCGA-A1-A0SE-01A-01-B_R2',
       'TCGA-A1-A0SF-01A-01-M_R1', 'TCGA-A1-A0SF-01A-01-M_R2',
       'TCGA-A1-A0SF-01A-01-T_R1', 'TCGA-A1-A0SF-01A-01-T_R2',
       ...
       'TCGA-S3-AA11-01A-03-T_R1', 'TCGA-S3-AA11-01A-03-T_R2',
       'TCGA-S3-AA12-01A-01-T_R1', 'TCGA-S3-AA12-01A-01-T_R2',
       'TCGA-S3-AA14-01A-01-T_R1', 'TCGA-S3-AA14-01A-01-T_R2',
       'TCGA-S3-AA15-01A-01-T_R1', 'TCGA-S3-AA15-01A-01-T_R2',
       'TCGA-S3-AA17-01A-01-T_R1', 'TCGA-S3-AA17-01A-01-T_R2'],
      dtype='object', name='patient_id', length=2519)